In [1]:
# notebook to illustrate co-occurrence graphs

In [2]:
# following only used for development, reloads the modules with any code changes
%load_ext autoreload
%autoreload 2

# inline matplotlib charts
%matplotlib inline

In [3]:
# import our text mining toolkit
import text_mining_toolkit as tmt

In [4]:
cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/recipes/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(directory_of_files="data_sets/mystery_corpus_01/txt/", text_filename_pattern="??.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/iraq_inquiry/txt/", text_filename_pattern="the-report*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/clinton_emails/txt/", text_filename_pattern="C0*.txt")
#cr = tmt.corpus_reader.CorpusReader(content_directory="data_sets/shakespeare_macbeth/txt/", text_filename_pattern="macbeth_act_0?_scene_0?.txt")

content_directory =  data_sets/recipes/txt/
text_filename_pattern =  ??.txt
self.documents populated =  22


In [5]:
# load previously calculated relevance scores
words_by_relevance = tmt.index_relevance.get_words_by_relevance(cr.content_directory)
# top most relevant
top_words_by_relevance = words_by_relevance[:100]

In [6]:
# first clear index
tmt.index_cooccurrence.delete_matrix(cr.content_directory)

# for all documents in corpus
for document_name in cr.get_documents():
    print("processing ", document_name)

    # get document text
    document_text = cr.get_text_by_document(document_name)

    # simplify whitespace (remove newlines)
    b = tmt.text_processing.simplify_whitespace(document_text)

    # only keep alphanumeric characters, removes punctuation
    c = tmt.text_processing.keep_only_alphanumeric(b)

    # make lowercase
    d = tmt.text_processing.to_lowercase(c)

    # split into words list
    dl = tmt.text_processing.split_text_into_words(d)
    # build n-grams
    #gl = tmt.word_processing.build_ngrams_from_words(dl,2)

    # remove stop words
    #el = tmt.word_processing.remove_stop_words(dl, "./stopwords/minimal-stop.txt")
    
    # remove all except those in the top most relevant list
    el = [word for word in dl if word in top_words_by_relevance]
    
    # update index
    tmt.index_cooccurrence.create_cooccurrence_matrix_for_document(cr.content_directory, document_name, el, window=4)
    pass


removed co-occurrence matrix file:  data_sets/recipes/txt/matrix.cooccurrence
processing  00.txt
processing  01.txt
processing  02.txt
processing  03.txt
processing  04.txt
processing  05.txt
processing  06.txt
processing  07.txt
processing  08.txt
processing  09.txt
processing  10.txt
processing  11.txt
processing  12.txt
processing  13.txt
processing  14.txt
processing  15.txt
processing  16.txt
processing  17.txt
processing  18.txt
processing  19.txt
processing  20.txt
processing  21.txt


In [7]:
# merge document indices into a corpus index
tmt.index_cooccurrence.merge_cooccurrence_matrices_for_corpus(cr.content_directory)

saving corpus co-occurrence matrix ...  data_sets/recipes/txt/matrix.cooccurrence


In [8]:
tmt.index_cooccurrence.print_matrix(cr.content_directory)

cooccurrence_matrix_file  data_sets/recipes/txt/matrix.cooccurrence
                 add  adding    ...        whole      with
add         0.000000     0.0    ...     0.000000  1.139566
adding      0.000000     0.0    ...     0.000000  0.778801
after       0.569783     0.0    ...     0.000000  1.509196
are         0.367879     0.0    ...     0.367879  1.875325
artichokes  0.000000     0.0    ...     0.000000  0.000000
as          0.000000     0.0    ...     0.569783  1.675172
asparagus   0.000000     0.0    ...     0.000000  1.348584
boiled      0.778801     0.0    ...     0.000000  1.348584
boiling     0.000000     0.0    ...     0.000000  0.367879
bread       0.000000     0.0    ...     0.000000  2.816488

[10 rows x 100 columns]


In [9]:
# query the co-occurrence value for a given pair of words
tmt.index_cooccurrence.query_cooccurance_matrix(cr.content_directory, "boiled", "with")

1.3485836078023279

In [10]:
# get the most likely next word by co-occurrence value for given word
tmt.index_cooccurrence.most_likely_next(cr.content_directory, "boiled")

'or'

In [11]:
#generate sentences based on co-occurence
sentence_length = 7

#initial words 
#for w in ["the", "once", "then", "olive", "tomato", "cut", "cook", "little", "boiled"]:
for w in ["olive"]:
    sentence = w
    for i in range(sentence_length):
        w = tmt.index_cooccurrence.most_likely_next(cr.content_directory, w)
        sentence += (" " + w)
        pass
    print(sentence)
    pass


olive oil or or or or or or


In [12]:
words_by_co_occurrence = tmt.index_cooccurrence.get_word_pairs_by_cooccurrence(cr.content_directory)

In [13]:
# 20 most common
words_by_co_occurrence[:20]

,word1,word2,weight
0,it,is,1.000000
1,grated,cheese,0.921939
2,with,butter,0.739343
3,or,or,0.717010
4,cut,them,0.673378
5,they,are,0.665214
6,them,with,0.638628
7,to,it,0.636051
8,with,or,0.559827
9,or,with,0.555309


In [14]:
# 20 least common
words_by_co_occurrence[-20:]

,word1,word2,weight
2856,this,which,0.022026
2857,egg,lemon,0.022026
2858,onion,make,0.022026
2859,eggs,crumbs,0.022026
2860,eggs,but,0.022026
2861,which,then,0.022026
2862,eggplants,that,0.022026
2863,eggplants,have,0.022026
2864,seasoning,some,0.022026
2865,egg,when,0.022026


In [15]:
# visualise co-occurrence of words
tmt.visualisation.plot_force_directed_graph(words_by_co_occurrence[:500])

force-directed graph

In [18]:
# visualise co-occurrence > 1
w = words_by_co_occurrence[words_by_co_occurrence['weight']>0.2]
tmt.visualisation.plot_force_directed_graph(w)

force-directed graph